In [1]:
import sys
sys.path.append('../')

from utils import Graph

graph = Graph('bolt://localhost:7687', 'neo4j', 'neo4jneo4j')

In [2]:
# Find positive examples
train_existing_links = graph.query_run_df("""
MATCH (n:Company)-[r:SUPPLIES]-(p:Company)
RETURN n.id AS node1, p.id AS node2, 1 AS label,r.revenue_pct AS revenue_pct
""",{})

In [3]:
train_existing_links.drop_duplicates(inplace = True)

In [23]:
# Find negative examples
nodes = list(train_existing_links['node1'].unique())

train_missing_links = graph.query_run_df("""
UNWIND $nodes AS node1
MATCH (n:Company) WHERE n.id = node1
MATCH (n:Company)-[r:SUPPLIES*1..2]-(p:Company) WHERE not((n:Company)-[:SUPPLIES]-(p:Company))
WITH n, collect(p) AS size 
RETURN n.id AS node1, size 
""", {"nodes" : nodes})

In [21]:
train_missing_links.drop_duplicates(inplace = True)
train_missing_links 

,node1,node2,label
0,132317,30132,0


In [4]:
def apply_graphy_features(data, rel_type):
    query = """
    UNWIND $pairs AS pair
    MATCH (p1) WHERE p1.id = pair.node1
    MATCH (p2) WHERE p2.id = pair.node2
    RETURN pair.node1 AS node1,
           pair.node2 AS node2,
           gds.alpha.linkprediction.commonNeighbors(
               p1, p2, {relationshipQuery: $relType}) AS cn,
           gds.alpha.linkprediction.preferentialAttachment(
               p1, p2, {relationshipQuery: $relType}) AS pa,
           gds.alpha.linkprediction.totalNeighbors(
               p1, p2, {relationshipQuery: $relType}) AS tn
    """
    pairs = [{"node1": node1, "node2": node2}  for node1,node2 in data[["node1", "node2"]].values.tolist()]
    params = {"pairs": pairs, "relType": rel_type}
    
    features = graph.run(query, params).to_data_frame()
    return pd.merge(data, features, on = ["node1", "node2"])